In [ ]:
import requests as re
from time import perf_counter_ns
from multiprocessing import Pool
import time
from matplotlib import pyplot as plt
import numpy as np

url = 'http://localhost:5000/auth'
batch_size = 25000
threads = 16
garbage_collector = 0

In [ ]:
def moving_average(data,window_size):
    window = np.ones(int(window_size))/float(window_size)
    return np.convolve(data, window, 'same')

def reject_outliers(data, m = 2.):
    data = np.array(data)
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else np.zeros(len(d))
    return data[s<m]
def plot_data(data):
    
    plt.plot(data)
    plt.title(f"Average {np.average(data)}")
    plt.show()

    rejected = reject_outliers(data,10)
    plt.plot(rejected)
    plt.title(f"Average {np.average(rejected)}")
    plt.show()
    


In [ ]:


def test_for_combination(login , password):
    times = []
    
    for x in range(batch_size):
        r = re.post(url, json={'username': login, 'password': password})
        time_taken = r.elapsed.microseconds
        times.append(time_taken)
    return np.array(times)

def test_for_combination_threaded(login , password):
    sum = 0
    with Pool(threads) as p:
        for x in range(batch_size):
            r = p.apply_async(re.post, args=(url,), kwds={'json': {'username': login, 'password': password}})
            time_taken = r.get().elapsed.microseconds
            sum += time_taken
    return sum*threads / batch_size


#print("good",test_for_combination_threaded('admin', 'ping2024'))
#print("good",test_for_combination('admin', 'ping2024'))
#print("bad",test_for_combination_threaded('gueuhre', 'ping2a24'))


In [ ]:

times_good = test_for_combination('user400', 'password')
times_bad = test_for_combination('admin', 'password')

plot_data(times_good)
plot_data(times_bad)

